In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import joblib
emb_matrix_train = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/sentipol_embedd_7410_128_768_alberto_training.joblib")
wmal_matrix_train = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/sentipolc_wmal_training_7410_128.joblib")

In [3]:
import pandas as pd
dataset_DF = pd.read_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/data/sentipolc_train_processed.csv")

In [4]:
from sklearn.preprocessing import OneHotEncoder
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = dataset_DF["oneg"]
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 0]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]
[1]


In [5]:
from keras.layers import Input, Dense, LSTM, Bidirectional, TimeDistributed, Flatten, Concatenate
!pip install keras-self-attention

In [6]:
from keras_self_attention import SeqWeightedAttention, SeqSelfAttention
from keras.layers import Add, Multiply

In [211]:
import keras
input_emb = Input(shape=(128,768))
input_wmal = Input(shape=(128,))

#Bi-LSTM
#bi = Bidirectional(LSTM(32, return_sequences=True))(input_emb)
bi = LSTM(64, return_sequences=True)(input_emb)
#att = SeqWeightedAttention()(bi)
att = SeqSelfAttention(attention_width=15,attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
                       attention_activation="sigmoid",
                       kernel_regularizer=keras.regularizers.l2(1e-8),#(1e-6),
                       bias_regularizer=keras.regularizers.l1(1e-8),#(1e-4),
                       attention_regularizer_weight=1e-8,#1e-4,
                       use_attention_bias=True,return_attention=False) (bi)

app = Multiply() ([bi, att])                      
flat = Flatten()(app)

#WMAL 
d1 = Dense(64) (input_wmal)

#Concat
concat = Concatenate(axis=1)([flat, d1])

d2 = Dense(32) (concat)
output = Dense(2,activation="softmax") (d2)

m = keras.Model(inputs=[input_emb,input_wmal], outputs=output)

opt = keras.optimizers.Adam(learning_rate=0.00005)
m.compile(loss="categorical_crossentropy",optimizer=opt,metrics=["accuracy"])


In [ ]:
m.summary()

In [212]:
#13 epochs for pos - 0.00002
#45 epochs for neg - 0.00005 - regularizers.l1(1e-4)
m.fit([emb_matrix_train,wmal_matrix_train],onehot_encoded,batch_size=128, epochs=45,validation_split=0.15)

Epoch 1/45
50/50 [==============================] - 5s 54ms/step - loss: 0.6821 - accuracy: 0.5739 - val_loss: 0.6077 - val_accuracy: 0.7968
Epoch 2/45
50/50 [==============================] - 2s 41ms/step - loss: 0.6231 - accuracy: 0.6826 - val_loss: 0.4938 - val_accuracy: 0.8345
Epoch 3/45
50/50 [==============================] - 2s 41ms/step - loss: 0.5672 - accuracy: 0.7157 - val_loss: 0.4338 - val_accuracy: 0.8543
Epoch 4/45
50/50 [==============================] - 2s 41ms/step - loss: 0.5293 - accuracy: 0.7380 - val_loss: 0.4111 - val_accuracy: 0.8543
Epoch 5/45
50/50 [==============================] - 2s 41ms/step - loss: 0.4974 - accuracy: 0.7551 - val_loss: 0.4041 - val_accuracy: 0.8462
Epoch 6/45
50/50 [==============================] - 2s 42ms/step - loss: 0.4779 - accuracy: 0.7688 - val_loss: 0.4060 - val_accuracy: 0.8336
Epoch 7/45
50/50 [==============================] - 2s 42ms/step - loss: 0.4506 - accuracy: 0.7882 - val_loss: 0.4310 - val_accuracy: 0.8138
Epoch 8/45
50

In [10]:
#emb_matrix_test = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/creagritrend_embedd_986_128_768_alberto.joblib")
#dataset_test = pd.read_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/data/creagritrend_processed.csv")
#wmal_matrix_test = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/creagritrend_wmal_agritrend_986_128.joblib")

emb_matrix_test = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/sentipol_embedd_2000_128_768_alberto_test.joblib")
dataset_test = pd.read_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/data/sentipolc_test_processed.csv")
wmal_matrix_test = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/sentipolc_wmal_test_2000_128.joblib")

In [213]:
# define example
data = dataset_test["oneg"]
values = array(data)
print(values)
# integer encode
integer_encoded_test = label_encoder.transform(values)
##print(integer_encoded_test)
# binary encode
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)
onehot_encoded_test = onehot_encoder.transform(integer_encoded_test)
#print(onehot_encoded_test)

[0 0 1 ... 1 1 0]


In [214]:
predictions = m.predict([emb_matrix_test,wmal_matrix_test])

In [193]:
print(predictions)

[[0.7390586  0.26094136]
 [0.94952714 0.0504729 ]
 [0.64912707 0.35087293]
 ...
 [0.00960974 0.9903902 ]
 [0.0710268  0.92897326]
 [0.5566742  0.44332585]]


In [215]:
import numpy as np
predictions_vec = np.zeros((2000,4))

i = 0
confidences= []
for item in predictions:
  index = np.argmax(item);
  confidences.append(item[index])
  predictions_vec[i][index] = 1
  i = i+1

labels_to_eval = []
for vec in predictions_vec:
  inverted = label_encoder.inverse_transform([argmax(vec)])[0]
  labels_to_eval.append(inverted)

In [216]:
!pip install scikit-learn
from sklearn.metrics import classification_report
print(classification_report(dataset_test["oneg"],labels_to_eval,digits=5))

              precision    recall  f1-score   support

           0    0.76585   0.86423   0.81207      1230
           1    0.72712   0.57792   0.64399       770

    accuracy                        0.75400      2000
   macro avg    0.74649   0.72107   0.72803      2000
weighted avg    0.75094   0.75400   0.74736      2000



In [217]:
#m.save("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/model_3_neg_sentipolc_0.72803.h5")

In [218]:
#dataset_test.to_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/predictions_SENTIPOLC_neg_test_model3_0.72803.csv");